In [1]:
import logging
import logging
import os
import pickle
from collections import namedtuple
from copy import deepcopy
from datetime import date
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
from torch.cuda.amp import GradScaler
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
import torch.nn as nn
from tqdm import tqdm
from dataset import GSNFrameDataset
from models import LikelihoodNetwork, proto_loss, exemplar_loss
from evaluation import model_evaluate, model_test

from GPUtil import showUtilization as gpu_usage
torch.manual_seed(232)


In [2]:
Config = namedtuple('parameters',
                    ['noun2idx',
                     'start_decade', 'end_decade',
                     'imagenet_embeddings',
                     'conceptnet_embeddings',
                     'histwords_embeddings',
                     'loss_function',
                     'hidden_dim_1', 'hidden_dim_2',
                     'vis_hidden_dim', 'ont_hidden_dim', 'ling_hidden_dim',
                     'modalities']
                    )


In [3]:
start_decade = 1850
end_decade = 1990
train_batch_size = 32
val_batch_size = 32
test_batch_size = 64
num_workers = 1
n_s = 8
n_q = 2
max_epochs_train = 200
max_epochs_eval = 10
eval_every = 20
vis_hidden_dim = 1000
ont_hidden_dim = 300
ling_hidden_dim = 300
hidden_dim_1 = 600
hidden_dim_2 = 400
loss_function = 'exemplar_loss'
learning_rate = 1e-4
weight_decay = 0
scheduler_step_size = 500
scheduler_gamma = 0.9
iters_to_accumulate = 1
grad_clip = 3.0
use_pretrained_model = False
cuda = True
log_dir = '/h/19/jadeleiyu/frame_extension/log/'
model_dir = '/h/19/jadeleiyu/frame_extension/models/'
prediction_dir = '/h/19/jadeleiyu/frame_extension/predictions/'
modalities = ['vis', 'ont', 'ling']

noun2idx = pickle.load(open('/h/19/jadeleiyu/frame_extension/data/gsn/support_noun2idx.p',
                            'rb'))  # a common noun2idx lookup dict shared across all modalities
list_converter = {'novel nouns': eval, 'novel nouns train': eval, 'novel nouns evaluation': eval,
                  'established nouns': eval, 'ground truth extended frames': eval,
                  'support nouns': eval, 'ground truth novel nouns': eval, 'support noun counts': eval,
                  'established noun counts': eval}

In [4]:
cuda_available = torch.cuda.is_available()
if cuda and cuda_available:
    device = torch.device("cuda:1")
    print("using gpu acceleration")
else:
    device = torch.device("cpu")
    print("using cpu")

using gpu acceleration


In [5]:
gpu_usage()

| ID | GPU | MEM |
------------------
|  0 |  0% | 53% |
|  1 |  0% | 10% |
|  2 |  0% | 13% |
|  3 |  0% | 10% |


In [6]:
print('performing learning by decade...')
for decade in range(start_decade, end_decade + 10, 10):
    print('begin learning in decade {}s'.format(decade))
    print('preparing learning data...')

    decade_training_df = pd.read_csv(
        '/h/19/jadeleiyu/frame_extension/data/gsn/training_df_{}s.csv'.format(decade),
        converters=list_converter)
    decade_training_df = decade_training_df.reset_index()
    decade_noun_test_df = pd.read_csv(
        '/h/19/jadeleiyu/frame_extension/data/gsn/noun_eval_df_{}s.csv'.format(decade),
        converters=list_converter)
    decade_frame_test_df = pd.read_csv(
        '/h/19/jadeleiyu/frame_extension/data/gsn/frame_eval_df_{}s.csv'.format(decade),
        converters=list_converter)
    decade_co_occurrence_mat = torch.load('/h/19/jadeleiyu/frame_extension/data/gsn/co_occurrence_mat_{}s.pt'
                                          .format(decade))

    train_ds = GSNFrameDataset(decade_training_df, noun2idx, n_q, n_s, learning_type='train')
    val_ds = GSNFrameDataset(decade_training_df, noun2idx, n_q, n_s, learning_type='val')
    noun_test_ds = GSNFrameDataset(decade_noun_test_df, noun2idx, n_q, n_s, learning_type='test_noun')
    frame_test_ds = GSNFrameDataset(decade_frame_test_df, noun2idx, n_q, n_s, learning_type='test_frame')

    train_loader = DataLoader(train_ds, shuffle=True, batch_size=train_batch_size, num_workers=num_workers)
    val_loader = DataLoader(val_ds, shuffle=True, batch_size=val_batch_size, num_workers=num_workers)
    noun_test_loader = DataLoader(noun_test_ds, shuffle=False, batch_size=test_batch_size, num_workers=num_workers)
    frame_test_loader = DataLoader(frame_test_ds, shuffle=False, batch_size=test_batch_size,
                                   num_workers=num_workers)

    conceptnet_embeddings = torch.tensor(pickle.load(
        open('/h/19/jadeleiyu/frame_extension/data/cnp/cnp_hist_embeddings_{}'.format(decade), 'rb')))
    histwords_embeddings = torch.tensor(np.load(
        '/h/19/jadeleiyu/frame_extension/data/histwords_embeddings/embeddings_{}.npy'.format(decade)))
    imagenet_embeddings = torch.load('/h/19/jadeleiyu/frame_extension/data/img/noun_image_features.pt')

    print('Generating configuration...')
    config = Config(
        noun2idx=noun2idx,
        start_decade=start_decade,
        end_decade=end_decade,
        conceptnet_embeddings=conceptnet_embeddings,
        histwords_embeddings=histwords_embeddings,
        imagenet_embeddings=imagenet_embeddings,
        loss_function=loss_function,
        vis_hidden_dim=vis_hidden_dim,
        ont_hidden_dim=ont_hidden_dim,
        ling_hidden_dim=ling_hidden_dim,
        hidden_dim_1=hidden_dim_1,
        hidden_dim_2=hidden_dim_2,
        modalities=modalities,
    )

    print('initializing model...')
    model = LikelihoodNetwork(config).to(device)
    if use_pretrained_model:
        model_fn = '{}_{}_{}.pt'.format(decade, loss_function, '-'.join(modalities))
        model.load_state_dict(torch.load(model_dir + model_fn))

    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = StepLR(optimizer, step_size=scheduler_step_size, gamma=scheduler_gamma)
    scaler = GradScaler()

    print('begin training...')
    # best_model = model_train(model, optimizer, scheduler, train_loader, val_loader, max_epochs, device)
    today = date.today()
    n_log = len([f for f in os.listdir(log_dir) if
                 os.path.isfile(os.path.join(log_dir, f)) and today.strftime("%b-%d-%Y") in f])
    log_fn = log_dir + today.strftime("%b-%d-%Y") + '_' + str(n_log) + '.log'
    logging.basicConfig(filename=log_fn, level=logging.DEBUG)
    logging.info(
        "Experiment {} on date {}. Decade: {}.".format(n_log, today.strftime("%b-%d-%Y"), decade) +
        " Loss function type: {}. Training Batch size: {}.".format(loss_function, train_batch_size) +
        " Evaluation Batch size: {}. Learning rate: {}. Modalities:{}. Joint hidden dim: {}."
        .format(val_batch_size, learning_rate, modalities, hidden_dim_2))

#     # evaluate the initialized model
#     eval_outputs = model_evaluate(model, val_loader, device, loss_function, max_epochs_eval)
#     logging.info(
#         "Decade {}, initialized model,".format(decade) +
#         " evaluation accuracy: {},".format(eval_outputs['acc'])
#     )

    best_eval_auc = 0
    best_epoch = 0
    best_model = deepcopy(model)
    for epoch in tqdm(range(max_epochs_train), position=0, leave=True):
        model.train()
        training_loss = 0
        for episode, (frame_idx_tensor, query_nouns_idx_tensor, support_nouns_idx_tensor,
                      support_nouns_total_counts_tensor) in tqdm(enumerate(train_loader), position=0, leave=True):

            query_nouns_idx_tensor, support_nouns_idx_tensor, support_nouns_total_counts_tensor \
                = query_nouns_idx_tensor.to(device), support_nouns_idx_tensor.to(device), \
                  support_nouns_total_counts_tensor.to(device)
            Hq_fused = model(query_nouns_idx_tensor)
            Hs_fused = model(support_nouns_idx_tensor)
            if loss_function == 'proto_loss':
                output = proto_loss(Hq_fused, Hs_fused)
            else:
                output = exemplar_loss(Hq_fused, Hs_fused)
            loss = output['loss']
            training_loss += loss.item()
            scaler.scale(loss).backward()
            if (episode + 1) % iters_to_accumulate == 0:
                # Adjust the learning rate based on the number of iterations.
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=grad_clip)
                scheduler.step()
                scaler.step(optimizer)
                # Updates the scale for next iteration.
                scaler.update()
                # Clear gradients
                optimizer.zero_grad()
        logging.info(
            "Decade {} Epoch {}, training loss: {},".format(decade, epoch + 1, training_loss / len(train_ds)))

        # evaluate model at the end of every 'eval_every' epochs
        if (epoch + 1) % eval_every == 0:
            test_results = model_test(model, noun_test_loader, frame_test_loader, device, decade_co_occurrence_mat,
                                      loss_function, topk=20)
            if test_results['auc_frame'] > best_eval_auc:
                best_eval_auc = test_results['auc_frame']
                best_epoch = epoch
                best_model = deepcopy(model)
                best_test_results = test_results
            logging.info(
                "Decade {} Epoch {},".format(decade, epoch + 1) +
                " mean auc score: {},".format(test_results['auc_frame'])
            )
    # log best evaluation results
    logging.info(
        "Decade {}, best evaluation auc score: {}, at epoch {}".format(
            decade, best_eval_auc,
            best_epoch))
#     # perform final prediction
#     best_test_results = model_test(best_model, noun_test_loader, frame_test_loader, device, decade_co_occurrence_mat,
#                loss_function, topk=20)
    # save prediction results for the last epoch
    logging.info(
        "Decade {}, best auc score for noun: {}; best auc score for frame: {}".format(
            decade, best_test_results['auc_noun'], best_test_results['auc_frame']))
    
    print('saving prediction results...')
    best_pred_df_noun = pd.DataFrame(best_test_results['noun_prediction_df'])
    best_pred_df_frame = pd.DataFrame(best_test_results['frame_prediction_df'])
    best_noun_precisions = best_test_results['precisions_noun']
    best_frame_precisions = best_test_results['precisions_frame']

    best_pred_df_noun.to_csv(prediction_dir + '{}_{}_{}_noun.csv'.format(decade, loss_function,
                                                                         '-'.join(modalities)))
    best_pred_df_frame.to_csv(prediction_dir + '{}_{}_{}_frame.csv'.format(decade, loss_function,
                                                                           '-'.join(modalities)))
    np.save(prediction_dir + '{}_{}_{}_noun_precisions.npy'.format(decade, loss_function,
                                                                         '-'.join(modalities)), best_noun_precisions)
    np.save(prediction_dir + '{}_{}_{}_frame_precisions.npy'.format(decade, loss_function,
                                                                         '-'.join(modalities)), best_frame_precisions)

    # save final model parameters
    model_fn = '{}_{}_{}_best.pt'.format(decade, loss_function, '-'.join(modalities))
    torch.save(best_model.state_dict(), model_dir + model_fn)


performing learning by decade...
begin learning in decade 1850s
preparing learning data...
Generating configuration...
initializing model...


  0%|          | 0/200 [00:00<?, ?it/s]

begin training...


0it [00:00, ?it/s]/h/19/jadeleiyu/frame_extension/frame_extension_venv/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
13it [00:00, 20.83it/s]
13it [00:00, 23.72it/s][00:00<02:12,  1.50it/s]
13it [00:00, 22.63it/s][00:01<02:08,  1.54it/s]
13it [00:00, 21.82it/s][00:01<02:07,  1.54it/s]
13it [00:00, 21.66it/s][00:02<02:07,  1.54it/s]
13it [00:00, 22.39it/s][00:03<02:07,  1.53it/s]
13it [00:00, 20.57it/s][00:03<02:06,  1.54it/s]
13it [00:00, 20.11it/s][00:04<02:08,  1.51it/s]
13it [00:00, 20.31it/s][00:05

13it [00:00, 19.02it/s]0 [02:24<00:48,  1.09it/s]
13it [00:00, 18.89it/s]0 [02:24<00:45,  1.15it/s]
13it [00:00, 20.13it/s]0 [02:25<00:42,  1.20it/s]
13it [00:00, 20.09it/s]0 [02:26<00:40,  1.25it/s]
13it [00:00, 21.83it/s]0 [02:27<00:38,  1.29it/s]
13it [00:00, 20.80it/s]0 [02:27<00:35,  1.35it/s]
13it [00:00, 21.78it/s]0 [02:28<00:34,  1.37it/s]
13it [00:00, 20.53it/s]0 [02:29<00:32,  1.40it/s]
13it [00:00, 20.36it/s]0 [02:29<00:32,  1.41it/s]
13it [00:00, 21.75it/s]0 [02:30<00:31,  1.41it/s]
13it [00:00, 19.36it/s]0 [02:31<00:29,  1.44it/s]
13it [00:00, 23.74it/s]0 [02:31<00:29,  1.42it/s]
13it [00:00, 19.77it/s]0 [02:32<00:27,  1.48it/s]
15it [00:00, 153.37it/s]
7it [00:00, 63.57it/s]
13it [00:00, 20.18it/s]0 [02:38<01:33,  2.34s/it]
13it [00:00, 18.53it/s]0 [02:39<01:12,  1.85s/it]
13it [00:00, 19.70it/s]0 [02:40<00:57,  1.52s/it]
13it [00:00, 18.89it/s]0 [02:40<00:47,  1.28s/it]
13it [00:00, 20.15it/s]0 [02:41<00:40,  1.12s/it]
13it [00:00, 19.73it/s]0 [02:42<00:34,  1.00it/s]
13

...

  0%|          | 0/200 [00:00<?, ?it/s]

begin learning in decade 1990s
preparing learning data...
Generating configuration...
initializing model...
begin training...


2it [00:00, 12.98it/s]
2it [00:00, 12.68it/s] [00:00<00:41,  4.83it/s]
2it [00:00, 14.26it/s] [00:00<00:41,  4.77it/s]
2it [00:00, 14.45it/s] [00:00<00:40,  4.88it/s]
2it [00:00, 13.59it/s] [00:00<00:41,  4.78it/s]
2it [00:00, 13.04it/s] [00:01<00:40,  4.84it/s]
2it [00:00, 12.84it/s] [00:01<00:41,  4.70it/s]
2it [00:00, 13.60it/s] [00:01<00:41,  4.70it/s]
2it [00:00, 13.48it/s] [00:01<00:40,  4.78it/s]
2it [00:00, 13.21it/s] [00:01<00:39,  4.81it/s]
2it [00:00, 13.04it/s]0 [00:02<00:39,  4.77it/s]
2it [00:00, 13.37it/s]0 [00:02<00:39,  4.74it/s]
2it [00:00, 13.64it/s]0 [00:02<00:39,  4.78it/s]
2it [00:00, 13.13it/s]0 [00:02<00:38,  4.82it/s]
2it [00:00, 12.28it/s]0 [00:02<00:38,  4.80it/s]
2it [00:00, 13.35it/s]0 [00:03<00:39,  4.70it/s]
2it [00:00, 13.48it/s]0 [00:03<00:38,  4.73it/s]
2it [00:00, 13.70it/s]0 [00:03<00:38,  4.78it/s]
2it [00:00, 12.73it/s]0 [00:03<00:37,  4.82it/s]
2it [00:00, 13.21it/s]0 [00:03<00:37,  4.78it/s]
6it [00:00, 78.63it/s]
1it [00:00, 13.74it/s]
2it [00:0

6it [00:00, 58.42it/s]
1it [00:00, 13.38it/s]
2it [00:00, 12.18it/s]00 [00:39<00:15,  2.66it/s]
2it [00:00, 13.09it/s]00 [00:40<00:12,  3.05it/s]
2it [00:00, 14.23it/s]00 [00:40<00:11,  3.44it/s]
2it [00:00, 13.46it/s]00 [00:40<00:09,  3.81it/s]
2it [00:00, 12.84it/s]00 [00:40<00:08,  4.02it/s]
2it [00:00, 11.29it/s]00 [00:40<00:08,  4.21it/s]
2it [00:00, 13.12it/s]00 [00:41<00:08,  4.25it/s]
2it [00:00, 14.89it/s]00 [00:41<00:07,  4.41it/s]
2it [00:00, 13.53it/s]00 [00:41<00:06,  4.66it/s]
2it [00:00, 13.17it/s]00 [00:41<00:06,  4.73it/s]
2it [00:00, 13.25it/s]00 [00:42<00:06,  4.77it/s]
2it [00:00, 12.07it/s]00 [00:42<00:06,  4.81it/s]
2it [00:00, 10.89it/s]00 [00:42<00:05,  4.73it/s]
2it [00:00, 13.37it/s]00 [00:42<00:06,  4.45it/s]
2it [00:00, 11.91it/s]00 [00:42<00:05,  4.59it/s]
2it [00:00, 11.34it/s]00 [00:43<00:05,  4.57it/s]
2it [00:00, 10.96it/s]00 [00:43<00:05,  4.40it/s]
2it [00:00, 11.78it/s]00 [00:43<00:05,  4.35it/s]
2it [00:00, 10.36it/s]00 [00:43<00:05,  4.24it/s]
2it 

saving prediction results...
